In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

Read in base overpass file ie ```overpass_input_final.xlsx``` and specify datetime formats for new columns (These are secondary overpasses that were appended to the original dataset, and this process must be apllied to any new column added).

In [2]:
# read in base file, must specify datetimes for those columns which aren't already in datetime format
overpass = pd.read_excel('csiro_ops.xlsx', index_col='Site')
overpass['landsat_8'] = pd.to_datetime(overpass.landsat_8, dayfirst=True, format='%d/%m/%Y %H:%M')
overpass['sentinel_2a'] = pd.to_datetime(overpass.sentinel_2a, dayfirst=True, format='%d/%m/%Y %H:%M')
overpass['sentinel_2b'] = pd.to_datetime(overpass.sentinel_2b, dayfirst=True, format='%d/%m/%Y %H:%M')
overpass['landsat_8_2'] = pd.to_datetime(overpass.landsat_8_2, dayfirst=True, format='%d/%m/%Y %H:%M')
overpass['sentinel_2a_2'] = pd.to_datetime(overpass.sentinel_2a_2, dayfirst=True, format='%d/%m/%Y %H:%M')
overpass['sentinel_2b_2'] = pd.to_datetime(overpass.sentinel_2b_2, dayfirst=True, format='%d/%m/%Y %H:%M')
overpass
#base file 

,Latitude,Longitude,Path,Row,landsat_8,landsat_8_2,sentinel_2a,sentinel_2a_2,sentinel_2b,sentinel_2b_2
Site,,,,,,,,,,
Blowering,-35.423731,148.258303,91,85,2020-01-07 23:48:41,NaT,2020-02-02 00:07:42,2020-01-29 23:58:16,2019-12-29 00:07:54,2020-04-01 23:58:26
Lake_Hume,-36.117778,147.035833,"91,92",85,2020-01-07 23:48:41,2019-12-29 23:54:36,2020-02-02 00:07:42,NaT,2019-12-29 00:07:54,NaT


In [3]:
# Input satellite timesteps for overpasses, calculated from https://evdc.esa.int/orbit/
l8_timestep = datetime.timedelta(days=15,hours=23,minutes=59,seconds=37)
s2a_timestep = datetime.timedelta(days=10,hours=0,minutes=1,seconds=10)
s2b_timestep = datetime.timedelta(days=9,hours=23,minutes=59,seconds=47)

In [4]:
l8_startdate = overpass['landsat_8']
l8_startdate_2 = overpass['landsat_8_2']
sentinel2a_startdate = overpass['sentinel_2a']
sentinel2a_2_startdate = overpass['sentinel_2a_2']
sentinel2b_startdate = overpass['sentinel_2b']
sentinel2b_2_startdate = overpass['sentinel_2b_2']

In [7]:
# Landsat 8 overpass prediction for 40 * the overpass frequency
landsat = list()
for i in range(40):
    landsat.append(l8_startdate + l8_timestep*(i))

landsat = pd.DataFrame(landsat)
#landsat = landsat + datetime.timedelta(hours=11) #convert to local time (Aus eastern daylight sdavings time) = utc + 11 hours

In [8]:
# Sentinel 2a overpass prediction for 25 * the overpass frequency, this is to give a similar total time to the L8 prediction
Sentinel_2A = []  
for i in range(50):
    Sentinel_2A.append(sentinel2a_startdate + s2a_timestep * (i))
    
Sentinel_2A = pd.DataFrame(Sentinel_2A)
#Sentinel_2A = Sentinel_2A + datetime.timedelta(hours=11)

In [9]:
# Sentinel 2b
Sentinel_2B = []
for i in range(50):
    Sentinel_2B.append(sentinel2b_startdate + s2b_timestep * (i))
    
Sentinel_2B = pd.DataFrame(Sentinel_2B)
#Sentinel_2B = Sentinel_2B + datetime.timedelta(hours=11)

In [10]:
# prediction for L8 overpasses at sites which are covered by more than 1 overpass in a 16-day period
# these sites are Dharawal, Dookie, Fowlers_Gap, Mullion & Winton 
# (dates from the Landsat acquisition tool, overpass times assumed to be similar to original overpasses for each site)
landsat_2 = []
for i in range(50):
    landsat_2.append(l8_startdate_2 + l8_timestep*(i))

landsat_2 = pd.DataFrame(landsat_2)
#landsat_2 = landsat_2 + datetime.timedelta(hours=11)

In [11]:
# Sentinel 2a secondary (Blowering)
Sentinel_2A_2 = []  
for i in range(50):
    Sentinel_2A_2.append(sentinel2a_2_startdate + s2a_timestep * (i))
    
Sentinel_2A_2 = pd.DataFrame(Sentinel_2A_2)
#Sentinel_2A_2 = Sentinel_2A_2 + datetime.timedelta(hours=11)

In [12]:
# Sentinel 2b secondary (Blowering)
Sentinel_2B_2 = []
for i in range(50):
    Sentinel_2B_2.append(sentinel2b_2_startdate + s2b_timestep * (i))
    
Sentinel_2B_2 = pd.DataFrame(Sentinel_2B_2)
#Sentinel_2B_2 = Sentinel_2B_2 + datetime.timedelta(hours=11)

In [13]:
# combine Landsat 8 data (base plus extra overpasses)
L8_combined = landsat.append(landsat_2)
drop_label_L8 = L8_combined.reset_index(drop=True)
L8_combined = drop_label_L8.sort_values(by='Lake_Hume') # specify first site which gets extra OP's to sort by
L8_combined.index.names = ['Landsat_8']
L8_combined

Site,Blowering,Lake_Hume
Landsat_8,,
40,NaT,2019-12-29 23:54:36
0,2020-01-07 23:48:41,2020-01-07 23:48:41
41,NaT,2020-01-14 23:54:13
1,2020-01-23 23:48:18,2020-01-23 23:48:18
42,NaT,2020-01-30 23:53:50
...,...,...
85,NaT,2021-12-18 23:37:21
86,NaT,2022-01-03 23:36:58
87,NaT,2022-01-19 23:36:35


In [14]:
# combine Sentinel 2A data (base plus extra overpasses)
S2A_combined = Sentinel_2A.append(Sentinel_2A_2)
drop_label_S2A = S2A_combined.reset_index(drop=True)
S2A_combined = drop_label_S2A.sort_values(by='Blowering') # specify first site which gets extra OP's to sort by
S2A_combined.index.names = ['Sentinel_2A']
S2A_combined

Site,Blowering,Lake_Hume
Sentinel_2A,,
50,2020-01-29 23:58:16,NaT
0,2020-02-02 00:07:42,2020-02-02 00:07:42
51,2020-02-08 23:59:26,NaT
1,2020-02-12 00:08:52,2020-02-12 00:08:52
52,2020-02-19 00:00:36,NaT
...,...,...
47,2021-05-17 01:02:32,2021-05-17 01:02:32
98,2021-05-24 00:54:16,NaT
48,2021-05-27 01:03:42,2021-05-27 01:03:42


In [15]:
# combine Sentinel 2B data 
S2B_combined = Sentinel_2B.append(Sentinel_2B_2)
drop_label_S2B = S2B_combined.reset_index(drop=True)
S2B_combined = drop_label_S2B.sort_values(by='Blowering') # specify first site which gets extra OP's to sort by
S2B_combined.index.names = ['Sentinel_2B']
S2B_combined

Site,Blowering,Lake_Hume
Sentinel_2B,,
0,2019-12-29 00:07:54,2019-12-29 00:07:54
1,2020-01-08 00:07:41,2020-01-08 00:07:41
2,2020-01-18 00:07:28,2020-01-18 00:07:28
3,2020-01-28 00:07:15,2020-01-28 00:07:15
4,2020-02-07 00:07:02,2020-02-07 00:07:02
...,...,...
95,2021-06-25 23:48:41,NaT
96,2021-07-05 23:48:28,NaT
97,2021-07-15 23:48:15,NaT


In [16]:
#write to .csv
L8_combined.to_excel('L8_overpasses_2020.xlsx')
S2A_combined.to_excel('S2A_overpasses_2020.xlsx')
S2B_combined.to_excel('S2B_overpasses_2020.xlsx')
#Sentinel_2B.to_excel('S2B_overpasses_2020.xlsx')